In [1]:
import numpy as np
import networkx as nx
import copy
import pickle
import sys
import time
sys.path.append("/Users/ziniuw/Desktop/research/Learned_QO/CE_scheme")
from BayesCard.Evaluation.utils import parse_query
from Schemas.stats.schema import gen_stats_light_schema
from Join_scheme.data_prepare import identify_key_values, process_stats_data

In [5]:
data_path = "/Users/ziniuw/Desktop/past_research/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
schema = gen_stats_light_schema(data_path)
all_keys, equivalent_keys = identify_key_values(schema)
print(equivalent_keys)

{'posts.Id': {'postLinks.RelatedPostId', 'comments.PostId', 'postHistory.PostId', 'posts.Id', 'votes.PostId', 'tags.ExcerptPostId', 'postLinks.PostId'}, 'users.Id': {'postHistory.UserId', 'comments.UserId', 'users.Id', 'badges.UserId', 'votes.UserId', 'posts.OwnerUserId'}}


In [3]:
bound_ensemble.bns

{'badges': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7ffa9802d4d0>,
 'votes': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7ffaec5a35d0>,
 'postHistory': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7ffaec6192d0>,
 'posts': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7ffaec619d10>,
 'users': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7ffaec6dced0>,
 'comments': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7ffaec737bd0>,
 'postLinks': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7ffaec858790>,
 'tags': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7ffaec87a1d0>}

In [ ]:
true_card = 17849233970
query57 = "SELECT COUNT(*) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.CommentCount>=0 AND p.CommentCount<=13 AND ph.PostHistoryTypeId=5 AND ph.CreationDate<='2014-08-13 09:20:10'::timestamp AND v.CreationDate>='2010-07-19 00:00:00'::timestamp AND b.Date<='2014-09-09 10:24:35'::timestamp AND u.Views>=0 AND u.DownVotes>=0 AND u.CreationDate>='2010-08-04 16:59:53'::timestamp AND u.CreationDate<='2014-07-22 15:15:22'::timestamp;"
t = time.time()
tables_all, table_queries, join_cond, join_keys = bound_ensemble.parse_query_simple(query57)
print(time.time()-t)

In [ ]:
t = time.time()
a = bound_ensemble.get_all_id_conidtional_distribution(table_queries, join_keys)
print(time.time()-t)
for table in a:
    print(a[table][0], np.sum(a[table][1]))


In [ ]:
print("join keys:", join_keys)

In [6]:
from Join_scheme.join_graph import process_condition, get_join_hyper_graph
equivalent_group = get_join_hyper_graph(join_keys, bound_ensemble.equivalent_keys)

NameError: name 'join_keys' is not defined

In [3]:
import numpy as np
import copy

from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from Join_scheme.data_prepare import identify_key_values
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform, construct_table_query


class Factor:
    """
    This the class defines a multidimensional conditional probability.
    """
    def __init__(self, variables, pdfs, equivalent_variables=[]):
        self.variables = variables
        self.equivalent_variables = equivalent_variables
        self.pdfs = pdfs
        self.cardinalities = dict()
        for i, var in enumerate(self.variables):
            self.cardinalities[var] = pdfs.shape[i]
            if len(equivalent_variables) != 0:
                self.cardinalities[equivalent_variables[i]] = pdfs.shape[i]


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, bns, table_buckets, schema):
        self.bns = bns
        self.table_buckets = table_buckets
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)

    def parse_query_simple(self, query):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        query = query.replace(" where ", " WHERE ")
        query = query.replace(" from ", " FROM ")
        query = query.replace(" and ", " AND ")
        query = query.split(";")[0]
        query = query.strip()
        tables_all = {}
        join_cond = []
        table_query = {}
        join_keys = {}
        tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
        for table_str in tables_str.split(","):
            table_str = table_str.strip()
            if " as " in table_str:
                tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
            else:
                tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

        # processing conditions
        conditions = query.split(" WHERE ")[-1].split(" AND ")
        for cond in conditions:
            table, cond, join, join_key = process_condition(cond, tables_all)
            if not join:
                attr = cond[0]
                op = cond[1]
                value = cond[2]
                if "Date" in attr:
                    assert "::timestamp" in value
                    value = timestamp_transorform(value.strip().split("::timestamp")[0])
                if table not in table_query:
                    table_query[table] = dict()
                construct_table_query(self.bns[table], table_query[table], attr, op, value)
            else:
                join_cond.append(cond)
                for tab in join_key:
                    if tab in join_keys:
                        join_keys[tab].add(join_key[tab])
                    else:
                        join_keys[tab] = set([join_key[tab]])

        return tables_all, table_query, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
            else:
                table_query = {}
            id_attrs, probs = self.bns[table].query_id_prob(table_query, key_attrs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res

    def eliminate_one_key_group_general(self, tables, key_group, factors):
        rest_groups = dict()
        rest_group_tables = dict()
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            for keys in temp:
                if keys in rest_groups:
                    assert factors[table].cardinalities[keys] == rest_groups[keys]
                    rest_group_tables[keys].append(table)
                else:
                    rest_groups[keys] = factors[table].cardinalities[keys]
                    rest_group_tables[keys] = [table]

        new_factor_variables = []
        new_factor_cardinalities = []
        for key in rest_groups:
            new_factor_variables.append(key)
            new_factor_cardinalities.append(rest_groups[key])
        new_factor_pdf = np.zeros(tuple(new_factor_cardinalities))

    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions"""
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))

        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for i in range(rest_group_cardinalty):
            for table in rest_group_tables:
                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    all_probs_eliminated.append(factors[table].pdfs[:, i])
                    all_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    all_probs_eliminated.append(factors[table].pdfs[:, i])
                    all_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    all_probs_eliminated.append(factors[table].pdfs[i, :])
                    all_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    all_probs_eliminated.append(factors[table].pdfs[i, :])
                    all_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes):
        all_probs = np.stack(all_probs, axis=0)
        all_modes = np.stack(all_modes, axis=0)
        multiplier = np.prod(all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        min_number = np.amin(all_probs[:, non_zero_idx]/all_modes[:, non_zero_idx], axis=0)
        multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
        return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality_bound(self, query_str):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution(table_queries, join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res

In [4]:
model_path = "/Users/ziniuw/Desktop/research/Learned_QO/CC_model/CE_scheme_models/stats/model_200.pkl"
with open(model_path, "rb") as f:
    new_BE = pickle.load(f)

In [ ]:
query_str = "SELECT COUNT(*) FROM users as u, comments as c, posts as p WHERE p.OwnerUserId = u.Id AND p.Id = c.PostId AND u.UpVotes>=0 AND u.CreationDate>='2010-08-21 21:27:38'::timestamp AND c.CreationDate>='2010-07-21 11:05:37'::timestamp AND c.CreationDate<='2014-08-25 17:59:25'::timestamp"
t = time.time()
res = new_BE.get_cardinality_bound(query57)
print(time.time() - t)
print(res)

In [6]:
query_file = "/Users/ziniuw/Desktop/past_research/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()


In [7]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = new_BE.get_cardinality_bound(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

estimating query 0: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.013953924179077148
estimating query 1: predicted 12028778.0, true_card 10220614, qerror 1.1769134417951799, latency 0.01628398895263672
estimating query 2: predicted 3935962.0, true_card 1458075, qerror 2.6994235550297483, latency 0.006045341491699219
estimating query 3: predicted 4017842.348968956, true_card 1709781, qerror 2.3499163629546453, latency 0.006907939910888672
estimating query 4: predicted 10505630.0, true_card 7491903, qerror 1.4022645514764407, latency 0.00669407844543457
estimating query 5: predicted 1028558.2624406518, true_card 428612, qerror 2.399742103442395, latency 0.00820302963256836
estimating query 6: predicted 59710469.99099071, true_card 55900138, qerror 1.068163194713235, latency 0.0065119266510009766
estimating query 7: predicted 11102.0, true_card 10972, qerror 1.0118483412322274, latency 0.004785060882568359
estimating query 8: predicted 60664815.0, true_card 699302, qerror 86.7

estimating query 57: predicted 323927.45338416565, true_card 288438, qerror 1.123040145140951, latency 0.008861064910888672
estimating query 58: predicted 34433.76851967809, true_card 31965, qerror 1.0772334903700325, latency 0.010083198547363281
estimating query 59: predicted 287950.77139124507, true_card 26836, qerror 10.73001831089749, latency 0.017393827438354492
estimating query 60: predicted 3663842.9422534136, true_card 2704241, qerror 1.3548507482333911, latency 0.004331111907958984
estimating query 61: predicted 34466.64273504273, true_card 32918, qerror 1.0470454685899122, latency 0.0035212039947509766
estimating query 62: predicted 88199.81426326046, true_card 86112, qerror 1.0242453347182792, latency 0.007119894027709961
estimating query 63: predicted 3400021.0872052265, true_card 2488080, qerror 1.366524021416203, latency 0.005769968032836914
estimating query 64: predicted 1580369.0, true_card 1056687, qerror 1.4955885706931191, latency 0.0024518966674804688
estimating que

estimating query 114: predicted 12529.436766792152, true_card 2112679, qerror 0.005930591806323702, latency 0.025020122528076172
estimating query 115: predicted 262028.97420841194, true_card 59978, qerror 4.368751445670278, latency 0.016777992248535156
estimating query 116: predicted 340237.31595774245, true_card 80977, qerror 4.201653753013108, latency 0.00894784927368164
estimating query 117: predicted 11723.786874193896, true_card 595820, qerror 0.0196767259813264, latency 0.02769184112548828
estimating query 118: predicted 2297101.0674555427, true_card 1187068, qerror 1.9351048696919997, latency 0.005403995513916016
estimating query 119: predicted 1580369.0, true_card 1056687, qerror 1.4955885706931191, latency 0.003083944320678711
estimating query 120: predicted 34773.0, true_card 34789, qerror 0.9995400845094714, latency 0.0017888545989990234
estimating query 121: predicted 2107529.2467487636, true_card 1486002, qerror 1.4182546502284408, latency 0.006443023681640625
estimating q

estimating query 187: predicted 59407.45769614781, true_card 41442, qerror 1.4335084623364656, latency 0.007872819900512695
estimating query 188: predicted 890.1416013634915, true_card 381, qerror 2.336329662371369, latency 0.008472204208374023
estimating query 189: predicted 70178.96580040909, true_card 57353, qerror 1.2236319948461125, latency 0.0024268627166748047
estimating query 190: predicted 100604154.60464251, true_card 91443369, qerror 1.100179878593958, latency 0.00978398323059082
estimating query 191: predicted 133368.94009331072, true_card 4204, qerror 31.724295930854122, latency 0.010444879531860352
estimating query 192: predicted 2046890.148581808, true_card 261931, qerror 7.814615866704621, latency 0.004250049591064453
estimating query 193: predicted 29960.80651338851, true_card 3236, qerror 9.258592865694842, latency 0.008852958679199219
estimating query 194: predicted 6070241.443035673, true_card 52313, qerror 116.03695913129955, latency 0.010574817657470703
estimating

estimating query 250: predicted 294968.9162187451, true_card 147464, qerror 2.000277465813657, latency 0.008394956588745117
estimating query 251: predicted 1281681.0, true_card 896180, qerror 1.4301602356669418, latency 0.009657144546508789
estimating query 252: predicted 1708790.1998566831, true_card 916826, qerror 1.8638107992756348, latency 0.0037267208099365234
estimating query 253: predicted 41278.445061728395, true_card 38775, qerror 1.0645633800574699, latency 0.005983114242553711
estimating query 254: predicted 990572.0, true_card 452527, qerror 2.188978779166768, latency 0.0015249252319335938
estimating query 255: predicted 70008.47017533022, true_card 60142, qerror 1.1640529110327262, latency 0.007472991943359375
estimating query 256: predicted 4637507.968920799, true_card 3728285, qerror 1.2438716377425005, latency 0.003432035446166992
estimating query 257: predicted 74077.34458867967, true_card 68676, qerror 1.0786496678414537, latency 0.005444049835205078
estimating query 

...

estimating query 2341: predicted 199981.3363832705, true_card 152614, qerror 1.3103734675932124, latency 0.03038477897644043
estimating query 2342: predicted 212578.01998091326, true_card 69462, qerror 3.0603498312878012, latency 0.004052877426147461
estimating query 2343: predicted 338824.073223793, true_card 303416, qerror 1.1166981082862901, latency 0.03015875816345215
estimating query 2344: predicted 1387797.1799698796, true_card 1326566, qerror 1.046157658171459, latency 0.03132176399230957
estimating query 2345: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.000888824462890625
estimating query 2346: predicted 277227.92131489416, true_card 18316, qerror 15.135833223132462, latency 0.005979299545288086
estimating query 2347: predicted 2756620.8809533296, true_card 361425, qerror 7.627089661626422, latency 0.006852149963378906
estimating query 2348: predicted 705831.3478479292, true_card 253217, qerror 2.787456402405562, latency 0.032774925231933594
estimating query 2349:

estimating query 2401: predicted 114059084.21422544, true_card 299574955, qerror 0.38073638103100255, latency 0.0416109561920166
estimating query 2402: predicted 114059084.21422544, true_card 3596066, qerror 31.71773938916178, latency 0.04126286506652832
estimating query 2403: predicted 5733078.574194326, true_card 11214638, qerror 0.5112138772731073, latency 0.012664794921875
estimating query 2404: predicted 6425281.411817149, true_card 42793812, qerror 0.1501451053675038, latency 0.03835582733154297
estimating query 2405: predicted 38723898.0, true_card 139632376, qerror 0.27732750175360477, latency 0.039594173431396484
estimating query 2406: predicted 114059084.21422544, true_card 319981016, qerror 0.35645578490889424, latency 0.039166927337646484
estimating query 2407: predicted 114059084.21422544, true_card 299574955, qerror 0.38073638103100255, latency 0.04114484786987305
estimating query 2408: predicted 160881.35853060134, true_card 29236, qerror 5.502851228984859, latency 0.002

estimating query 2464: predicted 6110707.296048014, true_card 21582838, qerror 0.2831280712966485, latency 0.018687963485717773
estimating query 2465: predicted 6110707.296048014, true_card 182241, qerror 33.53091398778548, latency 0.01516103744506836
estimating query 2466: predicted 18837459.123876993, true_card 1785176, qerror 10.552157951864126, latency 0.02019190788269043
estimating query 2467: predicted 35313884.00197089, true_card 96498890, qerror 0.36595119386317176, latency 0.019189119338989258
estimating query 2468: predicted 35313884.00197089, true_card 975631, qerror 36.195942935362744, latency 0.016138076782226562
estimating query 2469: predicted 18901047.3384418, true_card 17589523, qerror 1.0745628143777293, latency 0.02037978172302246
estimating query 2470: predicted 18806549.875649236, true_card 14185466, qerror 1.3257618660993749, latency 0.025424718856811523
estimating query 2471: predicted 108793440.07878518, true_card 524182774, qerror 0.20754867476584643, latency 0

estimating query 2535: predicted 1690976.530118479, true_card 265135, qerror 6.377794444786539, latency 0.0071871280670166016
estimating query 2536: predicted 9138589.47896956, true_card 1411723, qerror 6.473358781410772, latency 0.008089065551757812
estimating query 2537: predicted 213230.92406735267, true_card 12249, qerror 17.408027109752034, latency 0.006457090377807617
estimating query 2538: predicted 68407.29564180877, true_card 61687, qerror 1.108941845799095, latency 0.0096588134765625
estimating query 2539: predicted 10457.881282554443, true_card 10453, qerror 1.0004669743188026, latency 0.012018918991088867
estimating query 2540: predicted 329728.15690164577, true_card 175223, qerror 1.8817629928813326, latency 0.010321855545043945
estimating query 2541: predicted 69060.99913049958, true_card 38044, qerror 1.8152927959862153, latency 0.012928009033203125
estimating query 2542: predicted 294103.687513871, true_card 146996, qerror 2.0007597996807465, latency 0.01618885993957519

estimating query 2599: predicted 1562038.5536361681, true_card 245765, qerror 6.355821836454207, latency 0.013993263244628906
estimating query 2600: predicted 17448681.904901113, true_card 3429589, qerror 5.087688905259818, latency 0.015595197677612305
estimating query 2601: predicted 1176729.5581503639, true_card 383348, qerror 3.0696118361133067, latency 0.014874935150146484
estimating query 2602: predicted 31929709.323167656, true_card 1570429, qerror 20.331838830770227, latency 0.01596689224243164


In [14]:
qerror = np.asarray(qerror)

In [15]:
temp_qerror = copy.deepcopy(qerror)
temp_qerror[temp_qerror < 1] = 1/temp_qerror[temp_qerror < 1]

In [16]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")

q-error 50% percentile is 3.4119468314461705
q-error 90% percentile is 44.97340268580293
q-error 95% percentile is 128.56434745297838
q-error 99% percentile is 1347.9017899591543
q-error 100% percentile is 436888.8917196179


In [19]:
with open("stats_CEB_sub_queries_CE_scheme.txt", "w") as f:
    for p in pred:
        f.write(str(p)+"\n")

In [20]:
with open("stats_CEB_exec.sql", "r") as f:
    queries = f.readlines()

In [23]:
with open("stats_CEB_exec.sql", "w") as f:
    for q in queries:
        q = q.split("||")[-1]
        f.write(q)